## Search Engine With Tools and Agents

In [1]:
## arxiv - Research papers
## Tools creation
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

In [2]:
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper = api_wrapper_wiki)
wiki.name

'wikipedia'

In [3]:
## cretaing arxiv tool
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv.name

'arxiv'

In [4]:
tools = [wiki,arxiv]

In [5]:
from dotenv import load_dotenv
load_dotenv()
import os
os.environ["HUGGING_FACE_API_KEY"] = os.getenv("HUGGING_FACE_API_KEY")
from langchain_huggingface import HuggingFaceEmbeddings
hf_embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

c:\Users\bhanu prakash\Music\1.Langchain-Projects\3.Search Engine\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
## custom tools[RAG tool]
from langchain.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [7]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb = FAISS.from_documents(documents,hf_embeddings)
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001228542E380>, search_kwargs={})

In [8]:
## custom tool
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever,"langsmith_search","search any info about langsmith")

retriever_tool.name

'langsmith_search'

In [9]:
tools = [wiki,arxiv,retriever_tool]

In [10]:
## Run all the tools with agents and LLM models
## Tools,LLM --> AgentExecuter
from dotenv import load_dotenv
load_dotenv()
import os
groq_api_key = os.getenv("GROQ_API_KEY")

In [27]:
from langchain_groq import ChatGroq
llm = ChatGroq(groq_api_key = groq_api_key,model_name = "Llama3-8b-8192")

In [28]:
## Prompt Template 
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [29]:
## Agents
from langchain.agents import AgentExecutor,create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt=prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [30]:
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True)

In [31]:
agent_executor.invoke({"input": "What is LangSmith?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'What is LangSmith?'}`


Get started with LangSmith | ü¶úÔ∏èüõ†Ô∏è LangSmith

LangSmith is a platform for building production-grade LLM applications.
It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.
ObservabilityAnalyze traces in LangSmith and configure metrics, dashboards, alerts based on these.EvalsEvaluate your application over production traffic ‚Äî score application performance and get human feedback on your data.Prompt EngineeringIterate on prompts, with automatic version control and collaboration features.
LangSmith + LangChain OSSLangSmith is framework-agnostic ‚Äî¬†it can be used with or without LangChain's open source frameworks langchain and langgraph.If you are using either of these, you can enable LangSmith tracing with a single environment variable.
For more see the how-to guide for setting up LangSmith with LangChain or setti

{'input': 'What is LangSmith?',
 'output': "It seems like you've already gotten some information about LangSmith. Based on the provided text, LangSmith is a platform for building production-grade LLM applications, allowing developers to closely monitor and evaluate their applications. It provides features such as observability, evaluation, and prompt engineering to help developers ship their applications quickly and with confidence. LangSmith also claims to be framework-agnostic and can be used with or without LangChain's open-source frameworks."}

In [32]:
agent_executor.invoke({"input": "What is machine learning?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks witIt seems like the tool call for "knjetzfet" provided some relevant information about machine learning. Based on that, I can try to provide a more detailed answer.

Machine learning is a subset of artificial intelligence (AI) that involves training algorithms to learn from data and make predictions or decisions without being explicitly programmed. The algorithms can learn from experience and improve their performance over time, making them useful for a wide range of applications, such as image and speech recognition, natural language processing, and predictive modeling.

There are several types of machine learning, including supervised learn

{'input': 'What is machine learning?',
 'output': 'It seems like the tool call for "knjetzfet" provided some relevant information about machine learning. Based on that, I can try to provide a more detailed answer.\n\nMachine learning is a subset of artificial intelligence (AI) that involves training algorithms to learn from data and make predictions or decisions without being explicitly programmed. The algorithms can learn from experience and improve their performance over time, making them useful for a wide range of applications, such as image and speech recognition, natural language processing, and predictive modeling.\n\nThere are several types of machine learning, including supervised learning, unsupervised learning, and reinforcement learning. Supervised learning involves training an algorithm on labeled data, where the correct output is already known. Unsupervised learning involves training an algorithm on unlabeled data, where the goal is to discover patterns or structure in the

In [33]:
agent_executor.invoke({"input":"What's the paper 1706.03762 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on c
Invoking: `wikipedia` with `{'query': 'Attention Is All You Need'}`


Page: Attention Is All You Need
Summary: "Attention Is All You Need" is a 2017 landmark research paper in machine learning authored by eight scientists working at Google. The paper introduced a new deep learning architecture known as the transformer,The paper "Attention Is All You Need" introduced the transformer architecture, which is a type of recurrent neural network (RNN) that relies entirely on self-attention mechanisms. It was a significant departure from traditional RNNs that rely on recurrent connections and has since become widely used in many natural language processing 

{'input': "What's the paper 1706.03762 about?",
 'output': 'The paper "Attention Is All You Need" introduced the transformer architecture, which is a type of recurrent neural network (RNN) that relies entirely on self-attention mechanisms. It was a significant departure from traditional RNNs that rely on recurrent connections and has since become widely used in many natural language processing (NLP) applications.'}